In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# --- 1. Data Loading and Preprocessing ---
def load_and_preprocess(data_path):
    df = pd.read_csv(data_path)
    peaks, _ = find_peaks(df["MLII"], distance=150)
    heartbeats = []
    for peak in peaks:
        start = max(0, peak - 150)
        end = min(len(df), peak + 150)
        heartbeat = df["MLII"][start:end].values
        if len(heartbeat) == 300:
            heartbeats.append(heartbeat)
    heartbeats = np.array(heartbeats).reshape(-1, 300, 1)
    return heartbeats

def augment(heartbeat):
    noise = np.random.normal(0, 0.01, heartbeat.shape)
    time_shift = np.random.randint(-10, 10)
    augmented = tf.roll(heartbeat + noise, shift=time_shift, axis=0)
    return augmented

data_path = "C:/Users/abdulssekyanzi/EDA Dataset.csv/100.csv"
heartbeats = load_and_preprocess(data_path)

# --- 2. Model Architecture ---
def create_encoder(input_shape):
    model = models.Sequential([
        layers.Conv1D(64, 5, activation="relu", input_shape=input_shape),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation="relu"),
        layers.GlobalAveragePooling1D(),
        layers.Dense(128),
    ])
    return model

encoder = create_encoder((300, 1))

# NT-Xent Loss
def nt_xent_loss(embeddings, temperature=0.1):
    batch_size = tf.shape(embeddings)[0] // 2
    embeddings_a = embeddings[:batch_size]
    embeddings_b = embeddings[batch_size:]
    normalized_embeddings_a = tf.nn.l2_normalize(embeddings_a, axis=1)
    normalized_embeddings_b = tf.nn.l2_normalize(embeddings_b, axis=1)
    logits = tf.matmul(normalized_embeddings_a, tf.transpose(normalized_embeddings_b)) / temperature
    labels = tf.range(batch_size)
    loss_a = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    loss_b = tf.keras.losses.sparse_categorical_crossentropy(labels, tf.transpose(logits), from_logits=True)
    return (tf.reduce_mean(loss_a) + tf.reduce_mean(loss_b)) / 2

# --- 3. Training (Contrastive) ---
optimizer = optimizers.Adam(0.001)

@tf.function
def train_step(heartbeats_list):
    augmented_heartbeats_a = tf.stack([augment(hb) for hb in heartbeats_list])
    augmented_heartbeats_b = tf.stack([augment(hb) for hb in heartbeats_list])
    with tf.GradientTape() as tape:
        embeddings_a = encoder(augmented_heartbeats_a, training=True)
        embeddings_b = encoder(augmented_heartbeats_b, training=True)
        embeddings = tf.concat([embeddings_a, embeddings_b], axis=0)
        loss = nt_xent_loss(embeddings)
    gradients = tape.gradient(loss, encoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))
    return loss

epochs_contrastive = 10
batch_size = 32
for epoch in range(epochs_contrastive):
    for i in range(0, len(heartbeats), batch_size):
        batch = heartbeats[i:i + batch_size]
        loss = train_step(list(batch))
    print(f"Contrastive Epoch {epoch + 1}, Loss: {loss.numpy()}")

# --- 4. Pseudo-Labeling (Clustering) ---
embeddings = encoder.predict(heartbeats)
kmeans = KMeans(n_clusters=4, random_state=42)
pseudo_labels = kmeans.fit_predict(embeddings)

# --- 5. Supervised Fine-tuning (with Pseudo-Labels) ---
X_train_pseudo, X_test_pseudo, y_train_pseudo, y_test_pseudo = train_test_split(
    heartbeats, pseudo_labels, test_size=0.2, random_state=42
)

classification_head = models.Sequential([
    encoder,  # Use trained encoder
    layers.Dense(len(np.unique(pseudo_labels)), activation="softmax")
])
classification_head.compile(
    optimizer=optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

epochs_finetune = 10
classification_history = classification_head.fit(
    X_train_pseudo, y_train_pseudo, epochs=epochs_finetune, batch_size=32, validation_split=0.2
)

# --- 6. Evaluation ---
loss, accuracy = classification_head.evaluate(X_test_pseudo, y_test_pseudo)
print(f"Pseudo-Test Accuracy: {accuracy}")

# --- 7. Saving the model and KMeans ---
model = classification_head #set the model variable to the classification head
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)
with open('kmeans_model.pkl', 'wb') as file:
    pickle.dump(kmeans, file)
print("Models saved as pickle files")

C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Contrastive Epoch 1, Loss: 1.4179213047027588
Contrastive Epoch 2, Loss: 1.2786366939544678
Contrastive Epoch 3, Loss: 1.2307615280151367
Contrastive Epoch 4, Loss: 1.1858251094818115
Contrastive Epoch 5, Loss: 1.1683627367019653
Contrastive Epoch 6, Loss: 1.076168417930603
Contrastive Epoch 7, Loss: 1.061809778213501
Contrastive Epoch 8, Loss: 1.0177030563354492
Contrastive Epoch 9, Loss: 1.008847951889038
Contrastive Epoch 10, Loss: 0.9782706499099731
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step  
Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.5657 - loss: 1.2451 - val_accuracy: 0.5585 - val_loss: 1.0299
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5958 - loss: 0.9576 - val_accuracy: 0.6659 - val_loss: 0.7534
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7096 - loss: 0.7221 - val_accuracy: 0.7463 - val_loss: 0.6067
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7689 - loss: 0.6151 - val_accuracy: 0.8488 - val_loss:

In [ ]:
import pickle
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import threading

# Set your ngrok auth token here:
ngrok.set_auth_token("2tcplIa0ydyFY1vtebDXmVN2m3O_4We3ULzbCAWFk5LnWJk71")  # Replace with your actual auth token

# Load your trained model (the .pkl file you uploaded)
with open("model.pkl", "rb") as file:
    model = pickle.load(file)

# Define FastAPI app
app = FastAPI()

# Define the input data structure for the prediction
class Features(BaseModel):
    feature1: float
    feature2: float
    feature3: float
    feature4: float

@app.post("/predict/")
def predict(data: Features):
    features = np.array([[data.feature1, data.feature2, data.feature3, data.feature4]])
    prediction = model.predict(features)
    return {"prediction": int(prediction[0])}

# Function to run uvicorn in a separate thread
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start uvicorn in a separate thread
uvicorn_thread = threading.Thread(target=run_uvicorn)
uvicorn_thread.start()

# Start ngrok to expose the API publicly
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

# Keep the main thread alive (optional)
uvicorn_thread.join()

INFO:     Started server process [18564]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://87ed-102-134-149-102.ngrok-free.app
INFO:     102.134.149.102:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.134.149.102:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.134.149.102:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.134.149.102:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.134.149.102:0 - "GET / HTTP/1.1" 404 Not Found


In [ ]:
# app.py (Create this file in the same directory as your notebook)
import pickle
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import tensorflow as tf
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

# Load the models
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

# Preprocessing functions (from your original code)
def load_and_preprocess(heartbeat_signal):
    df = pd.DataFrame({'MLII': heartbeat_signal})
    peaks, _ = find_peaks(df["MLII"], distance=150)
    heartbeats = []
    for peak in peaks:
        start = max(0, peak - 150)
        end = min(len(df), peak + 150)
        heartbeat = df["MLII"][start:end].values
        if len(heartbeat) == 300:
            heartbeats.append(heartbeat)
    heartbeats = np.array(heartbeats).reshape(-1, 300, 1)
    return heartbeats

class HeartbeatInput(BaseModel):
    heartbeat_signal: list

@app.post('/predict')
async def predict(heartbeat_input: HeartbeatInput):
    try:
        heartbeat_signal = heartbeat_input.heartbeat_signal
        heartbeats = load_and_preprocess(heartbeat_signal)
        if heartbeats.size == 0:
            raise HTTPException(status_code=400, detail="No heartbeats detected.")
        prediction = model.predict(heartbeats)
        return {'prediction': prediction.argmax(axis=1).tolist()}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
# In your Jupyter Notebook:
!pip install fastapi uvicorn pyngrok tensorflow pandas scikit-learn scipy

from pyngrok import ngrok
import uvicorn
import threading

ngrok.set_auth_token("2tcplIa0ydyFY1vtebDXmVN2m3O_4We3ULzbCAWFk5LnWJk71") #add your auth token.

def run_uvicorn():
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

uvicorn_thread = threading.Thread(target=run_uvicorn)
uvicorn_thread.start()

public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")